In [9]:
import spacy 
import pyArango
import os
from os import path
import time
import glob
import pandas as pd
from arango import ArangoClient

Lecture du texte depuis le chemin

In [2]:
def get_text(path):
    with open(path, encoding='utf8') as f:
        return(f.read().replace('\n',' '))
        f.close()

In [3]:
dir_path = os.getcwd()
dir_path

'/home/paul/projects/text explorer'

Liste des fichiers présents dans le dossier choisi pour l'analyse

In [4]:
files = glob.glob('/home/paul/projects/text_for_app/*.{}'.format('txt'))
files

# Intégrer la lecture de fichiers dans une logique de vérification des informations dans la base de données

Obtention du nom du fichier depuis le chemin

In [5]:
def get_filename_from_path(path):
    return os.path.normpath(path).split(os.sep)[-1]

In [6]:
documents = pd.DataFrame({'filepath':files,
                          'doc_name':[get_filename_from_path(filepath) for filepath in files],
                          'doc_number':list(range(0,len(files)))})
documents

,filepath,doc_name,doc_number
0,/home/paul/projects/text_for_app/jean_blog.txt,jean_blog.txt,0
1,/home/paul/projects/text_for_app/emploi étudia...,emploi étudiant et inégalités sociales.txt,1


Initialisation de la connection avec la base de données 

In [61]:
# Initialize the client for ArangoDB.
client = ArangoClient(hosts="http://localhost:8529")

In [62]:
sys_db = client.db('_system', username='root',password='root')

si besoin de repartir à zero:

In [330]:
sys_db.delete_database('text')

True

Check si la base de données existe déjà et création si besoin.

In [331]:
if not sys_db.has_database('text'):
    
    sys_db.create_database('text')
    db = client.db("text", username="root", password="root")
    graph = db.create_graph('text_explorer')
    # création des collections
    docs = graph.create_vertex_collection('docs')
    sentences = graph.create_vertex_collection('sentences')
    tokens = graph.create_vertex_collection('tokens')
    lemmas = graph.create_vertex_collection('lemmas')
    # création des arrêtes
    is_from = graph.create_edge_definition(
        edge_collection='is_from',
        from_vertex_collections=['sentences'],
        to_vertex_collections=['docs']
    )
    contracts_to = graph.create_edge_definition(
        edge_collection='contracts_to',
        from_vertex_collections=['tokens'],
        to_vertex_collections=['lemmas']
    )
    syntagmatic_link = graph.create_edge_definition(
        edge_collection='syntagmatic_link',
        from_vertex_collections=['tokens'],
        to_vertex_collections=['tokens']
    )
else:
    db = client.db("text", username="root", password="root")

L'objectif maintenant est de remplir ces champs avec les données extraites depuis le texte

### insertion des documents

In [332]:
for doc_name, number, path  in zip(documents['doc_name'], documents['doc_number'], documents['filepath']):
    docs.insert({'_key':f'doc{number}',
                 'doc_name':doc_name,
                 'doc_path':path,
                 'doc_number':number,
                 'processed':'False'})

In [333]:
in_db_doclist = list(db.aql.execute('''FOR doc in docs RETURN doc.doc_name'''))
in_db_doclist

['jean_blog.txt', 'emploi étudiant et inégalités sociales.txt']

Chargement du modèle

In [334]:
nlp = spacy.load('fr_core_news_lg')

Traitement des fichiers par le modèle

## se renseigner sur comment faire du traitement en batch
    - mesurer l'empreinte mémoire de l'opération et ajuster combien de documents en même temps peuvent être traités

In [335]:
texts_to_process_df = pd.DataFrame(
                        list(
                            db.aql.execute('''
                                            FOR doc in docs
                                            FILTER doc.processed == 'False'
                                            RETURN {path :doc.doc_path, number : doc.doc_number}
                                            ''')
                            )
                        )
texts_to_process_df

,path,number
0,/home/paul/projects/text_for_app/jean_blog.txt,0
1,/home/paul/projects/text_for_app/emploi étudia...,1


### insertion des phrases
Si nous construisons le ID sur la base de la boucle ci-dessous comment incrémenter l'ID avec des traitements batch ? 
- stocker le numéro de chaque documents dans la db 
- avant chaque insertion requêter le numéro de document le plus élevé et repartir de ce point

In [336]:
processed_docs = list(nlp.pipe([get_text(path) for path in texts_to_process_df['path']]))

In [337]:
for doc, doc_number in zip(processed_docs,texts_to_process_df['number']):
    for sentence_number, sentence in enumerate(doc.sents):
        if sentence.text != ' ':
            sentences.insert({'_key':f'doc{doc_number}sent{sentence_number}',
                              'content':sentence.text})
        else:
            pass

Insertion des tokens et lemmas

Récupérer le vocabulaire token et lemma de chaque doc et les insérer dans la db

- clé = nombre arbitraire d'insertion
- valeur = le token ou lemma

Pour insérer de nouveaux mots par la suite faire une requette sur la table des tokens / lemma et insérer ceux qui ne sont pas déjà présents 

pour les relations faire une requete par phrase où l'on récupère chaque mot et sa clé associée
- relier ensuite les mots entre eux en utilisant les clés et la modalité de cette relation

Comment relier les phrases aux documents et les mots aux phrases ? 

In [338]:
new_vocab_tokens = []
new_vocab_lemmas = []
seen_tokens = set()
seen_lemmas = set()

In [339]:
for token in processed_docs[1]:
    if token.text.lower() not in seen_tokens  and not token.is_punct and not token.is_stop and not token.is_space:
        new_vocab_tokens.append(token.text.lower())
    seen_tokens.add(token.text.lower())

In [340]:
for token in processed_docs[1]:
    if token.lemma_ not in seen_lemmas  and not token.is_punct and not token.is_stop and not token.is_space:
        new_vocab_tokens.append(token.lemma_.lower())
    seen_tokens.add(token.text.lower())

In [ ]:
new_vocab_tokens

Check si il faut commencer un indexage pour le vocabulaire ou le récupérer et poursuivre

In [ ]:
vocab = list(db.aql.execute("""
            FOR doc in tokens
            RETURN {token :doc.token, key : doc._key}
            """))
vocab

In [274]:
if len(vocab) == 0:
        for count, token in enumerate(new_vocab_tokens):
            tokens.insert({"_key":f'token{count}',
                           "number":count,
                           "token":token})
else :
    pass

In [342]:
dict_list_tokens_to_insert = []
for count, token in enumerate(new_vocab_tokens):
    dict_list_tokens_to_insert.append({"_key":f'token{count}',
                                       'token':token})

# faire un bulk import pour toutes les opérations 
- finir d'importer les lemmas
- faire un import des liens entre les documents

In [291]:
with db.begin_batch_execution() as batch_token:
    for count, token in enumerate(new_vocab_tokens):
            tokens.insert({"_key":f'token{count}',
                           "number":count,
                           "token":token})
    batch_token.commit()

In [296]:
pd.DataFrame({'token':new_vocab_tokens,
              '_key':list(range(0,len(new_vocab_tokens)))}).to_csv('vocab_export.csv')

In [ ]:
for doc in processed_docs:
    for count, sentence in enumerate(doc.sents):
        print(count, sentence.text, sentence.has_vector)

' 71 \x0c'

###

Extraction d'une forme tabulaire de la structure syntagmatique.
  > Le processus est itéré sur l'ensemble des phrases du document

on obtient une liste de dataframes contenant la structure syntagmatique pour chaque phrase

In [18]:
def create_dependancy_df_list(processed_text):
    df_list = []
    for sentence in processed_text.sents:
        token_text, token_dep, token_head_text, token_head_pos = [], [], [], []
        for token in sentence:
            if not token.is_punct and not token.is_stop and not token.is_space:
                token_text.append(token.text)
                token_dep.append(token.dep_), 
                token_head_text.append(token.head.text), 
                token_head_pos.append( token.head.pos_)
        df = pd.DataFrame({'token':token_text,
                           'dep':token_dep,
                           'head_text':token_head_text,
                           'head_pos':token_head_pos})    
        if not df.empty:
            df_list.append(df)
        else:
            pass
    return df_list

In [19]:
file_1_processed = nlp(get_text(documents['filepath'][0]))
file_2_processed = nlp(get_text(documents['filepath'][1]))

In [20]:
create_dependancy_df_list(file_1_processed)[0]

,token,dep,head_text,head_pos
0,faire,xcomp,vais,VERB
1,inutilement,advmod,faire,VERB
2,durer,xcomp,faire,VERB
3,suspense,obj,durer,VERB
4,réponse,nsubj,oui,ADV
5,oui,parataxis,vais,VERB
